In [ ]:
#| default_exp parse

# parse

> Parsing stuff

In [ ]:
#| export
import pathlib
import urllib.parse
import datetime

import pandas as pd

import sproc.xml
import sproc.hier

## Sample data

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.cwd().parent / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples')

A (sample) file in that directory

In [ ]:
xml_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert xml_file.exists()
xml_file

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

XML file is read into a **hierarchical** (*column-multiindex*) `pd.DataFrame`. **<font color='red'>CAVEAT:</font>** a hierarchical `pd.DataFrame` is assumed.

In [ ]:
df = sproc.hier.flat_df_to_multiindexed_df(sproc.xml.to_curated_df(xml_file))
df.head()

id  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  https://contrataciondelestado.es/sindicacion/P...   
1  https://contrataciondelestado.es/sindicacion/P...   
2  https://contrataciondelestado.es/sindicacion/P...   
3  https://contrataciondelestado.es/sindicacion/P...   
4  https://contrataciondelestado.es/sindicacion/P...   

                                             summary  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  Id licitación: C. 2-2021; Órgano de Contrataci...   
1  Id licitación: 8128_3/2021; Órgano de Contrata...   
2  Id licitación: 1000_0005-CP01-2021-000063; Órg...   
3  Id licitación: 1379/2020 4738; Órgano de Contr...   
4  Id licitación: 2021-44; Órgano de Contratación...   

                                               title  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  L'objecte del contracte és la renovació de tot...   
1  Obras de restauración hidromorfológica del río...   
2  Contrato del servicio de realización de labore...   
3  Obres de renovació de l'enllumenat públic a la...   
4  Subministre i la instal·lació fotovoltaica en ...   

                           updated        ContractFolderStatus  \
                                              ContractFolderID   
                                                                 
                                                                 
                                                                 
                                                                 
0 2022-01-03 00:11:41.826000+00:00                   C. 2-2021   
1 2022-01-03 00:00:11.194000+00:00                 8128_3/2021   
2 2022-01-03 00:00:10.399000+00:00  1000_0005-CP01-2021-000063   
3 2022-01-02 23:11:40.740000+00:00              1379/2020 4738   
4 2022-01-02 23:11:40.696000+00:00                     2021-44   

                                                                               \
  ContractFolderStatusCode                            LocatedContractingParty   
                                                            BuyerProfileURIID   
                                                                                
                                                                                
                                                                                
0                      ADJ  https://contractaciopublica.gencat.cat/ecofin_...   
1                      PUB                                               <NA>   
2                       EV                                               <NA>   
3                       EV  https://contractaciopublica.gencat.cat/ecofin_...   
4                       EV  https://contractaciopublica.gencat.cat/ecofin_...   

                                                      \
                                                       
                                               Party   
                                           PartyName   
                                                Name   
                                                       
0                           Ajuntament de Sant Ramon   
1                             Pleno del Ayuntamiento   
2  El Director General de Comunicación y Relacion...   
3                         Ajuntament de Canet de Mar   
4                   

The column to discriminate regions

In [ ]:
#| export
domain_discriminative_columns_paths = [
    ['ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID'],
    ['ContractFolderStatus', 'LegalDocumentReference', 'Attachment', 'ExternalReference', 'URI']
]

It is turned

In [ ]:
domain_discriminative_columns = [sproc.hier.pad_col_levels(df, p) for p in domain_discriminative_columns_paths]
domain_discriminative_columns

[('ContractFolderStatus',
  'LocatedContractingParty',
  'BuyerProfileURIID',
  '',
  '',
  ''),
 ('ContractFolderStatus',
  'LegalDocumentReference',
  'Attachment',
  'ExternalReference',
  'URI',
  '')]

Unique values of the *first* one

In [ ]:
unique_values = df[domain_discriminative_columns[0]].unique().tolist()
unique_values[:5]

['https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=2763318',
 <NA>,
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=29178875',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=8530338',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=16054009']

`pd.NA` values are filtered out

In [ ]:
unique_values = list(filter(pd.notna, unique_values))
unique_values[:5]

['https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=2763318',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=29178875',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=8530338',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=16054009',
 'https://contractaciopublica.gencat.cat/ecofin_pscp/AppJava/cap.pscp?reqCode=viewDetail&idCap=15937468']

A one-liner returning a `pd.DataFrame` of columns with *parsed* domains

In [ ]:
domains = df[domain_discriminative_columns].applymap(lambda x: urllib.parse.urlparse(x).netloc if pd.notna(x) else pd.NA)
domains

ContractFolderStatus                                
            LocatedContractingParty          LegalDocumentReference
                  BuyerProfileURIID                      Attachment
                                                  ExternalReference
                                                                URI
                                                                   
0    contractaciopublica.gencat.cat                            <NA>
1                              <NA>                            <NA>
2                              <NA>                            <NA>
3    contractaciopublica.gencat.cat  contractaciopublica.gencat.cat
4    contractaciopublica.gencat.cat  contractaciopublica.gencat.cat
..                              ...                             ...
112                             NaN                             NaN
113                             NaN                             NaN
114                             NaN                             NaN
115                             NaN                             NaN
116                             NaN                             NaN

[117 rows x 2 columns]

How many *non-nulls* are in every column?

In [ ]:
domains.notna().sum()

ContractFolderStatus  LocatedContractingParty  BuyerProfileURIID                              42
                      LegalDocumentReference   Attachment         ExternalReference  URI      70
dtype: int64

What about the combination?

In [ ]:
domains[domain_discriminative_columns[0]].combine_first(domains[domain_discriminative_columns[1]]).notna().sum()

77

A function returning a `pd.Series` with domains

In [ ]:
#| export
def domain(
    df: pd.DataFrame # Input
    ) -> pd.Series: # Domains
    "Extract the (internet) domains from the given data"
    
    # columns names from "path"s
    columns = [sproc.hier.pad_col_levels(df, p) for p in domain_discriminative_columns_paths]
    
    domains = df[columns].applymap(lambda x: urllib.parse.urlparse(x).netloc if pd.notna(x) else pd.NA)
    
    # the result is initialized with the first column of domains...
    res = domains[columns[0]]
    
    # ...and  the remaining...
    for c in columns[1:]:

        # ...are used to update it
        res = res.combine_first(domains[c])
    
    return res

The function adds a new column to the `pd.DataFrame` *inplace*

In [ ]:
domain(df)

0      contractaciopublica.gencat.cat
1                                <NA>
2                                <NA>
3      contractaciopublica.gencat.cat
4      contractaciopublica.gencat.cat
                    ...              
112                              <NA>
113                              <NA>
114                              <NA>
115                              <NA>
116                              <NA>
Name: (ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , ), Length: 117, dtype: object

## Dates

A function to parse a string containing either a year or a year **and** a month

In [ ]:
#| export
def year_and_maybe_month(
    s: str # Raw date
    ) -> datetime.datetime: # Parsed date

    try:

        # year and month
        d = datetime.datetime.strptime(s, '%Y%d')
    
    except ValueError:

        # only year
        d = datetime.datetime.strptime(s, '%Y').replace(month=12)

    return d


Only a year

In [ ]:
year_and_maybe_month('2021')

datetime.datetime(2021, 12, 1, 0, 0)

A year and a month

In [ ]:
year_and_maybe_month('202205')

datetime.datetime(2022, 1, 5, 0, 0)

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('50_parse.ipynb')